In [ ]:
import cv2
from ultralytics import YOLO
from config import YOLO_MODEL_PATH, YOLO_CLASSES, CONFIDENCE_THRESHOLD


model = YOLO(YOLO_MODEL_PATH)

def detect_objects(frame):
    detections = []
    results = model(frame)
    for detection in results[0].boxes:
        cls_id = int(detection.cls[0])
        label = YOLO_CLASSES[cls_id]
        conf = float(detection.conf[0])
        if conf > CONFIDENCE_THRESHOLD:
            x1, y1, x2, y2 = map(int, detection.xyxy[0])
            detections.append({
                "label": label,
                "confidence": conf,
                "box": (x1, y1, x2, y2)
            })
    return detections

# Path to the folder containing the 10 test images
image_folder = "knife/"
image_paths = [f"{image_folder}image{i+1}.jpg" for i in range(10)]

# Process each image
for idx, image_path in enumerate(image_paths):
    # Load the image
    frame = cv2.imread(image_path)
    if frame is None:
        print(f"Error loading image: {image_path}")
        continue
    
    # Detect objects
    detections = detect_objects(frame)
    
    # Draw bounding boxes and labels on the image
    for detection in detections:
        label = detection['label']
        confidence = detection['confidence']
        x1, y1, x2, y2 = detection['box']
        
        # Draw bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Prepare label text with confidence
        label_text = f"{label} ({confidence:.2f})"
        
        # Put label text above the bounding box
        cv2.putText(frame, label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Display the image with bounding boxes
    cv2.imshow(f"Detections for Image {idx + 1}", frame)
    cv2.waitKey(0)  # Press any key to proceed to the next image
    cv2.destroyAllWindows()

print("Processing complete.")


0: 384x640 1 knife, 253.8ms
Speed: 18.3ms preprocess, 253.8ms inference, 14.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 knife, 276.6ms
Speed: 13.0ms preprocess, 276.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Error loading image: knife/image3.jpg



[ WARN:0@15773.993] global loadsave.cpp:241 findDecoder imread_('knife/image3.jpg'): can't open/read file: check file path/integrity


0: 384x640 1 knife, 3 persons, 216.5ms
Speed: 2.1ms preprocess, 216.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 320x640 1 celurit, 299.9ms
Speed: 4.5ms preprocess, 299.9ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 640)
Error loading image: knife/image6.jpg
Error loading image: knife/image7.jpg
Error loading image: knife/image8.jpg
Error loading image: knife/image9.jpg
Error loading image: knife/image10.jpg
Processing complete.


[ WARN:0@15799.988] global loadsave.cpp:241 findDecoder imread_('knife/image6.jpg'): can't open/read file: check file path/integrity
[ WARN:0@15799.990] global loadsave.cpp:241 findDecoder imread_('knife/image7.jpg'): can't open/read file: check file path/integrity
[ WARN:0@15799.992] global loadsave.cpp:241 findDecoder imread_('knife/image8.jpg'): can't open/read file: check file path/integrity
[ WARN:0@15799.993] global loadsave.cpp:241 findDecoder imread_('knife/image9.jpg'): can't open/read file: check file path/integrity
[ WARN:0@15799.993] global loadsave.cpp:241 findDecoder imread_('knife/image10.jpg'): can't open/read file: check file path/integrity


In [13]:
import cv2
import tensorflow as tf
import numpy as np
from tensorflow.lite.python.interpreter import Interpreter
from config import TFLITE_MODEL_PATH

# Load and initialize the TFLite model
interpreter = Interpreter(model_path=TFLITE_MODEL_PATH)
interpreter.allocate_tensors()
runner = interpreter.get_signature_runner()

# Initialize states for the TFLite model
init_states = {
    name: tf.zeros(x['shape'], dtype=x['dtype']).numpy()
    for name, x in runner.get_input_details().items()
}
del init_states['image']

def detect_violence(frame, states):
    processed_frame = tf.image.convert_image_dtype(frame, tf.float32)
    processed_frame = tf.image.resize_with_pad(processed_frame, 172, 172)
    clip = tf.expand_dims(processed_frame, axis=0)
    
    outputs = runner(**states, image=clip)
    logits = outputs.pop('logits')[0]
    states = {key: value for key, value in outputs.items()}
    
    # Calculate softmax probabilities
    probs = tf.nn.softmax(logits).numpy()
    return probs, states

# Path to the input and output video
input_video_path = "/Users/bintangrestubawono/Downloads/Equal Rights, Equal Left! When Guys Fight Back Compilation 2024 - TLS - Fighting Spirit (720p, h264, youtube).mp4"  # Replace with your video path
output_video_path = "output_video_with_annotations.mp4"

# Open the input video
cap = cv2.VideoCapture(input_video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Start processing the video
states = init_states
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert BGR (OpenCV format) to RGB for TensorFlow
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Run violence detection
    probs, states = detect_violence(rgb_frame, states)
    fight_prob, no_fight_prob = probs[0] * 100, probs[1] * 100

    # Overlay the probabilities on the frame
    overlay_text = f"Violence: {fight_prob:.2f}%  |  No Violence: {no_fight_prob:.2f}%"
    cv2.putText(frame, overlay_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Write the frame with annotation to the output video
    out.write(frame)

# Release resources
cap.release()
out.release()
print(f"Annotated video saved to {output_video_path}")

Annotated video saved to output_video_with_annotations.mp4


In [22]:
import os
import cv2
import tensorflow as tf
import numpy as np
from tensorflow.lite.python.interpreter import Interpreter
from sklearn.metrics import classification_report, accuracy_score
from config import TFLITE_MODEL_PATH

# Load and initialize the TFLite model
interpreter = Interpreter(model_path=TFLITE_MODEL_PATH)
interpreter.allocate_tensors()
runner = interpreter.get_signature_runner()

# Initialize states for the TFLite model
init_states = {
    name: tf.zeros(x['shape'], dtype=x['dtype']).numpy()
    for name, x in runner.get_input_details().items()
}
del init_states['image']

def detect_violence(frame, states):
    processed_frame = tf.image.convert_image_dtype(frame, tf.float32)
    processed_frame = tf.image.resize_with_pad(processed_frame, 172, 172)
    clip = tf.expand_dims(processed_frame, axis=0)
    
    outputs = runner(**states, image=clip)
    logits = outputs.pop('logits')[0]
    states = {key: value for key, value in outputs.items()}
    
    # Calculate softmax probabilities
    probs = tf.nn.softmax(logits).numpy()
    return probs, states

# Path to the dataset
dataset_path = "/Users/bintangrestubawono/Documents/Violence Detection/datasets/RWF-2000/val/"  # Folder containing subfolders "Fight" and "NoFight"
categories = ["No_Fight", "Fight"]

# Initialize storage for predictions and true labels
all_predictions = []
all_true_labels = []

for category in categories:
    category_path = os.path.join(dataset_path, category)
    label = 1 if category == "Fight" else 0  # 1 for Fight, 0 for NoFight

    for video_file in os.listdir(category_path):
        video_path = os.path.join(category_path, video_file)
        
        # Open the video file
        cap = cv2.VideoCapture(video_path)
        states = init_states
        frame_predictions = []
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Convert BGR (OpenCV format) to RGB for TensorFlow
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Run violence detection
            probs, states = detect_violence(rgb_frame, states)
            fight_prob = probs[0]  # Probability of violence

            # Append frame-level prediction (1 for Violence if >0.8, else 0 for No Violence)
            frame_predictions.append(1 if fight_prob > 0.6 else 0)

        # Release video resources
        cap.release()

        # Determine video-level prediction by majority voting
        if len(frame_predictions) > 0:
            video_prediction = 1 if sum(frame_predictions) > (len(frame_predictions) / 2) else 0
        else:
            video_prediction = 0  # Default to NoFight if no frames were processed

        # Append video-level prediction and true label
        all_predictions.append(video_prediction)
        all_true_labels.append(label)

# Calculate accuracy and classification report
accuracy = accuracy_score(all_true_labels, all_predictions)
report = classification_report(all_true_labels, all_predictions, target_names=["NOT VIOLENCE", "VIOLENCE"])

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)


Accuracy: 0.74
Classification Report:
              precision    recall  f1-score   support

NOT VIOLENCE       0.67      0.95      0.79       200
    VIOLENCE       0.92      0.53      0.67       200

    accuracy                           0.74       400
   macro avg       0.79      0.74      0.73       400
weighted avg       0.79      0.74      0.73       400



In [27]:
import cv2
from ultralytics import YOLO
from config import YOLO_MODEL_PATH, YOLO_CLASSES, CONFIDENCE_THRESHOLD

# Load YOLO model
model = YOLO(YOLO_MODEL_PATH)

def calculate_iou(box1, box2):
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])

    intersection_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - intersection_area

    return intersection_area / union_area if union_area != 0 else 0

def non_max_suppression(detections, iou_threshold):
    detections = sorted(detections, key=lambda x: x["confidence"], reverse=True)
    suppressed_boxes = []
    while detections:
        chosen_box = detections.pop(0)
        suppressed_boxes.append(chosen_box)
        detections = [box for box in detections if calculate_iou(chosen_box["box"], box["box"]) < iou_threshold]
    return suppressed_boxes

def detect_objects(frame):
    detections = []
    results = model(frame)
    for detection in results[0].boxes:
        cls_id = int(detection.cls[0])
        label = YOLO_CLASSES[cls_id]
        conf = float(detection.conf[0])
        if conf > CONFIDENCE_THRESHOLD:
            x1, y1, x2, y2 = map(int, detection.xyxy[0])
            detections.append({
                "label": label,
                "confidence": conf,
                "box": (x1, y1, x2, y2)
            })
    return detections

def main(image_path):
    # Load image
    frame = cv2.imread(image_path)
    
    # Detect objects
    detections = detect_objects(frame)
    
    # Filter detections for "People"
    filtered_detections = [d for d in detections if d['label'] == 'person']
    
    # Apply Non-Max Suppression
    iou_threshold = 0.5  # Adjust this threshold as needed
    suppressed_detections = non_max_suppression(filtered_detections, iou_threshold)
    
    # Draw boxes and labels on the image
    for det in suppressed_detections:
        x1, y1, x2, y2 = det['box']
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Draw rectangle
        cv2.putText(frame, f"{det['label']} {det['confidence']:.2f}", (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
    # Display the result
    cv2.imshow("Detections", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Specify your image path
image_path = "/Users/bintangrestubawono/Downloads/istockphoto-1457298134-612x612.jpg"  # Replace with your image path
main(image_path)



0: 384x640 11 persons, 239.3ms
Speed: 12.1ms preprocess, 239.3ms inference, 15.2ms postprocess per image at shape (1, 3, 384, 640)


In [28]:
import cv2
from ultralytics import YOLO
from config import YOLO_MODEL_PATH, YOLO_CLASSES, CONFIDENCE_THRESHOLD

# Load YOLO model
model = YOLO(YOLO_MODEL_PATH)

def calculate_iou(box1, box2):
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])

    intersection_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - intersection_area

    return intersection_area / union_area if union_area != 0 else 0

def non_max_suppression(detections, iou_threshold):
    detections = sorted(detections, key=lambda x: x["confidence"], reverse=True)
    suppressed_boxes = []
    
    while detections:
        chosen_box = detections.pop(0)
        suppressed_boxes.append(chosen_box)
        
        detections = [box for box in detections if calculate_iou(chosen_box["box"], box["box"]) < iou_threshold]
    
    return suppressed_boxes

def detect_objects(frame):
    detections = []
    results = model(frame)
    for detection in results[0].boxes:
        cls_id = int(detection.cls[0])
        label = YOLO_CLASSES[cls_id]
        conf = float(detection.conf[0])
        if conf > CONFIDENCE_THRESHOLD:
            x1, y1, x2, y2 = map(int, detection.xyxy[0])
            detections.append({
                "label": label,
                "confidence": conf,
                "box": (x1, y1, x2, y2)
            })
    return detections

def main(image_path):
    # Load image
    frame = cv2.imread(image_path)
    
    # Detect objects
    detections = detect_objects(frame)
    
    # Filter detections for "People"
    filtered_detections = [d for d in detections if d['label'] == 'person']
    
    # Apply Non-Max Suppression
    iou_threshold = 0.5  # Adjust this threshold as needed
    suppressed_detections = non_max_suppression(filtered_detections, iou_threshold)
    
    # Label groups
    group_detections = []
    for det in suppressed_detections:
        group_detections.append({
            "label": "group",  # Labeling as 'group'
            "confidence": det['confidence'],
            "box": det['box']
        })

    # Draw boxes and labels on the image
    for det in group_detections:
        x1, y1, x2, y2 = det['box']
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Draw rectangle
        cv2.putText(frame, f"{det['label']} {det['confidence']:.2f}", (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
    # Display the result
    cv2.imshow("Detections", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Specify your image path
image_path = "/Users/bintangrestubawono/Downloads/istockphoto-1457298134-612x612.jpg"  # Replace with your image path
main(image_path)


0: 384x640 11 persons, 247.5ms
Speed: 15.9ms preprocess, 247.5ms inference, 11.3ms postprocess per image at shape (1, 3, 384, 640)


In [32]:
import cv2
from ultralytics import YOLO
from config import YOLO_MODEL_PATH, YOLO_CLASSES, CONFIDENCE_THRESHOLD

# Load YOLO model
model = YOLO(YOLO_MODEL_PATH)

def calculate_iou(box1, box2):
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])

    intersection_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - intersection_area

    return intersection_area / union_area if union_area != 0 else 0

def group_overlapping_boxes(detections, iou_threshold):
    grouped_boxes = []
    while detections:
        base_box = detections.pop(0)
        group = [base_box]
        
        # Group all detections that have IoU >= threshold with the base box
        i = 0
        while i < len(detections):
            if calculate_iou(base_box["box"], detections[i]["box"]) >= iou_threshold:
                group.append(detections.pop(i))
            else:
                i += 1
                
        # Determine if this is a group or a single person
        if len(group) > 1:
            # Calculate the bounding box that covers the entire group
            x1 = min(box["box"][0] for box in group)
            y1 = min(box["box"][1] for box in group)
            x2 = max(box["box"][2] for box in group)
            y2 = max(box["box"][3] for box in group)
            confidence = max(box["confidence"] for box in group)
            
            grouped_boxes.append({
                "label": "group",
                "confidence": confidence,
                "box": (x1, y1, x2, y2)
            })
        else:
            # Single person detection, keep original box and label
            grouped_boxes.append({
                "label": "person",
                "confidence": base_box["confidence"],
                "box": base_box["box"]
            })
        
    return grouped_boxes

def detect_objects(frame):
    detections = []
    results = model(frame)
    for detection in results[0].boxes:
        cls_id = int(detection.cls[0])
        label = YOLO_CLASSES[cls_id]
        conf = float(detection.conf[0])
        if conf > CONFIDENCE_THRESHOLD:
            x1, y1, x2, y2 = map(int, detection.xyxy[0])
            detections.append({
                "label": label,
                "confidence": conf,
                "box": (x1, y1, x2, y2)
            })
    return detections

def main(image_path):
    # Load image
    frame = cv2.imread(image_path)
    
    # Detect objects
    detections = detect_objects(frame)
    
    # Filter detections for "People"
    filtered_detections = [d for d in detections if d['label'] == 'person']
    
    # Group overlapping bounding boxes
    iou_threshold = 0.2  # Threshold to consider boxes as overlapping
    grouped_detections = group_overlapping_boxes(filtered_detections, iou_threshold)
    
    # Draw boxes and labels on the image
    for det in grouped_detections:
        x1, y1, x2, y2 = det['box']
        color = (0, 255, 0) if det['label'] == 'person' else (255, 0, 0)  # Green for person, Blue for group
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)  # Draw rectangle
        cv2.putText(frame, f"{det['label']} {det['confidence']:.2f}", (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    # Display the result
    cv2.imshow("Detections", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Specify your image path
image_path = "/Users/bintangrestubawono/Downloads/istockphoto-1457298134-612x612.jpg"  # Replace with your image path
main(image_path)



0: 384x640 11 persons, 225.1ms
Speed: 1.5ms preprocess, 225.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)


: 

In [3]:
!pip install SpeechRecognition pydub


    extract-msg (<=0.29.*)
                 ~~~~~~~^

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install ffmpeg-python


    extract-msg (<=0.29.*)
                 ~~~~~~~^

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install aiortc opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.0/896.0 kB 916.8 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 799.0 kB/s eta 0:00:00a 0:00:01
    extract-msg (<=0.29.*)
                 ~~~~~~~^
  Attempting uninstall: pyopenssl
    Found existing installation: pyOpenSSL 23.2.0
    Uninstalling pyOpenSSL-23.2.0:
      Successfully uninstalled pyOpenSSL-23.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nevermined-sdk-py 0.14.0 requires contracts-lib-py==1.0.0, but you have contracts-lib-py 1.0.4 which is incompatible.
nevermined-sdk-py 0.14.0 requires web3==5.26.0, but you have web3 5.31.3 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [7]:
import asyncio
import cv2
import numpy as np
from aiortc import RTCPeerConnection, RTCSessionDescription, MediaStreamTrack
from aiortc.contrib.signaling import TcpSocketSignaling
from av import VideoFrame
from datetime import datetime, timedelta

class VideoReceiver:
    def __init__(self):
        self.track = None

    async def handle_track(self, track):
        print("Inside handle track")
        self.track = track
        frame_count = 0
        while True:
            try:
                print("Waiting for frame...")
                frame = await asyncio.wait_for(track.recv(), timeout=5.0)
                frame_count += 1
                print(f"Received frame {frame_count}")
                
                if isinstance(frame, VideoFrame):
                    print(f"Frame type: VideoFrame, pts: {frame.pts}, time_base: {frame.time_base}")
                    frame = frame.to_ndarray(format="bgr24")
                elif isinstance(frame, np.ndarray):
                    print(f"Frame type: numpy array")
                else:
                    print(f"Unexpected frame type: {type(frame)}")
                    continue
              
                 # Add timestamp to the frame
                current_time = datetime.now()
                new_time = current_time - timedelta( seconds=55)
                timestamp = new_time.strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
                cv2.putText(frame, timestamp, (10, frame.shape[0] - 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.imwrite(f"imgs/received_frame_{frame_count}.jpg", frame)
                print(f"Saved frame {frame_count} to file")
                cv2.imshow("Frame", frame)
    
                # Exit on 'q' key press
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            except asyncio.TimeoutError:
                print("Timeout waiting for frame, continuing...")
            except Exception as e:
                print(f"Error in handle_track: {str(e)}")
                if "Connection" in str(e):
                    break
        print("Exiting handle_track")
async def run(pc, signaling):
    await signaling.connect()

    @pc.on("track")
    def on_track(track):
        if isinstance(track, MediaStreamTrack):
            print(f"Receiving {track.kind} track")
            asyncio.ensure_future(video_receiver.handle_track(track))

    @pc.on("datachannel")
    def on_datachannel(channel):
        print(f"Data channel established: {channel.label}")

    @pc.on("connectionstatechange")
    async def on_connectionstatechange():
        print(f"Connection state is {pc.connectionState}")
        if pc.connectionState == "connected":
            print("WebRTC connection established successfully")

    print("Waiting for offer from sender...")
    offer = await signaling.receive()
    print("Offer received")
    await pc.setRemoteDescription(offer)
    print("Remote description set")

    answer = await pc.createAnswer()
    print("Answer created")
    await pc.setLocalDescription(answer)
    print("Local description set")

    await signaling.send(pc.localDescription)
    print("Answer sent to sender")

    print("Waiting for connection to be established...")
    while pc.connectionState != "connected":
        await asyncio.sleep(0.1)

    print("Connection established, waiting for frames...")
    await asyncio.sleep(100)  # Wait for 35 seconds to receive frames

    print("Closing connection")

async def main():
    signaling = TcpSocketSignaling("https://7319-202-145-7-5.ngrok-free.app/", 3000)
    pc = RTCPeerConnection()
    
    global video_receiver
    video_receiver = VideoReceiver()

    try:
        await run(pc, signaling)
    except Exception as e:
        print(f"Error in main: {str(e)}")
    finally:
        print("Closing peer connection")
        await pc.close()
if __name__ == "__main__":
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop